<a href="https://colab.research.google.com/github/elangbijak4/Blockchain_Research/blob/main/Rev3_Framework_Mini1_Infrastruktur_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import hashlib
import time
import json
import random
import os

In [43]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    @staticmethod
    def calculate_hash(index, previous_hash, timestamp, data):
        value = f"{index}{previous_hash}{timestamp}{data}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    @staticmethod
    def create_genesis_block():
        return Block(0, "0", int(time.time()), "Genesis Block", Block.calculate_hash(0, "0", int(time.time()), "Genesis Block"))

    def to_dict(self):
        return {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "data": self.data,
            "hash": self.hash
        }

### Protokol Proof of Elapsed Time

In [44]:
class ProofOfElapsedTime:
    def __init__(self):
        self.wait_times = {}

    def wait_for_block(self, address):
        wait_time = random.uniform(0, 10)  # Simulating wait time
        self.wait_times[address] = wait_time
        time.sleep(wait_time)

    def select_validator(self):
        return min(self.wait_times, key=self.wait_times.get)

### Protokol Proof of Burn

In [45]:
class ProofOfBurn:
    def __init__(self):
        self.burned_coins = {}  # To keep track of burned coins by address

    def burn_coins(self, address, amount):
        if address in self.burned_coins:
            self.burned_coins[address] += amount
        else:
            self.burned_coins[address] = amount

    def select_validator(self):
        total_burned = sum(self.burned_coins.values())
        pick = random.uniform(0, total_burned)
        current = 0
        for address, burned in self.burned_coins.items():
            current += burned
            if current > pick:
                return address

### Protokol Proof of ByzantineFaultTolerance

In [47]:
class ByzantineFaultTolerance:
    def __init__(self, validators, faulty_validators=None):
        self.validators = validators
        self.faulty_validators = faulty_validators if faulty_validators else []
        self.leader = None

    def select_leader(self):
        non_faulty_validators = [v for v in self.validators if v not in self.faulty_validators]
        self.leader = random.choice(non_faulty_validators)
        return self.leader

    def select_validator(self, chain_length):
        non_faulty_validators = [v for v in self.validators if v not in self.faulty_validators]
        return random.choice(non_faulty_validators)

    def validate_block(self, block, chain):
        previous_block = chain[block.index - 1]
        if block.previous_hash != previous_block.hash:
            return False
        if block.hash != block.calculate_hash():
            return False
        return True

    def add_faulty_validator(self, validator):
        if validator not in self.faulty_validators:
            self.faulty_validators.append(validator)

    def remove_faulty_validator(self, validator):
        if validator in self.faulty_validators:
            self.faulty_validators.remove(validator)

    def reach_consensus(self, block, chain):
        votes = {validator: self.validate_block(block, chain) for validator in self.validators if validator not in self.faulty_validators}
        positive_votes = sum(votes.values())
        if positive_votes > len(self.validators) * 2 / 3:
            return True
        return False

    def calculate_reward(self, validator):
        # For simplicity, we set a fixed reward. In practice, it might depend on various factors.
        return 10

### Protokol Proof of Stake

In [48]:
class ProofOfStake:
    def __init__(self):
        self.stakeholders = {}  # To keep track of stakeholder's stakes

    def add_stakeholder(self, address, stake):
        if address in self.stakeholders:
            self.stakeholders[address] += stake
        else:
            self.stakeholders[address] = stake

    def select_validator(self):
        total_stake = sum(self.stakeholders.values())
        pick = random.uniform(0, total_stake)
        current = 0
        for address, stake in self.stakeholders.items():
            current += stake
            if current > pick:
                return address

In [49]:
class Blockchain:
    def __init__(self, consensus_protocol, filename='blockchain.json'):
        self.filename = filename
        self.consensus_protocol = consensus_protocol
        self.chain = self.load_chain()

    def load_chain(self):
        if os.path.exists(self.filename):
            with open(self.filename, 'r') as file:
                data = json.load(file)
                return [Block(**block) for block in data]
        else:
            return [Block.create_genesis_block()]

    def save_chain(self):
        with open(self.filename, 'w') as file:
            json.dump([block.to_dict() for block in self.chain], file, indent=4)

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        new_block = Block(index, previous_hash, timestamp, data, "")

        if isinstance(self.consensus_protocol, ProofOfBurn):
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoB")
        if isinstance(self.consensus_protocol, ProofOfStake):
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoS")
        if isinstance(self.consensus_protocol, ByzantineFaultTolerance):
            validator = self.consensus_protocol.select_validator(len(self.chain))
            print(f"Block #{index} created by {validator} using BFT")

        elif isinstance(self.consensus_protocol, ProofOfElapsedTime):
            for address in self.consensus_protocol.wait_times.keys():
                self.consensus_protocol.wait_for_block(address)
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoET")

        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)
        self.save_chain()

    def to_json(self):
        self.save_chain()
        with open(self.filename, 'r') as file:
            return file.read()

In [50]:
class SmartContract:
    def __init__(self, code):
        self.code = code
        self.storage = {}

    def execute(self, blockchain, **kwargs):
        local_context = {
            'blockchain': blockchain,
            'storage': self.storage
        }
        local_context.update(kwargs)
        exec(self.code, {}, local_context)
        self.storage = local_context['storage']

In [51]:
class ERC721:
    def __init__(self):
        self.tokens = {}
        self.token_owners = {}
        self.token_metadata = {}
        self._token_owner = {}  # tokenId => owner
        self._owned_tokens = {}  # owner => list of tokenIds
        self._token_approvals = {}  # tokenId => approved address
        self._operator_approvals = {}  # owner => (operator => approved)

    def _mint(self, to, tokenId, metadata):
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to
        self.token_metadata[tokenId] = metadata

    def _transfer(self, from_, to, tokenId):
        self._owned_tokens[from_].remove(tokenId)
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to

    def mint(self, to, metadata):
        tokenId = hashlib.sha256(metadata.encode()).hexdigest()
        self._mint(to, tokenId, metadata)
        return tokenId

    def transfer(self, from_, to, tokenId):
        self._transfer(from_, to, tokenId)

In [52]:
class ERC20:
    def __init__(self):
        self.balances = {}
        self.total_supply = 0

    def mint(self, to, amount):
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.total_supply += amount

    def transfer(self, from_, to, amount):
        if from_ not in self.balances or self.balances[from_] < amount:
            raise ValueError("Insufficient balance")
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.balances[from_] -= amount

    def balance_of(self, address):
        return self.balances.get(address, 0)

In [53]:
class MyBlockchain(Blockchain):
    def __init__(self,pos):
        super().__init__(pos)
        self.smart_contracts = {}
        self.nft_contract = ERC721()
        self.fungible_token_contract = ERC20()
        self.pos = pos

    def store_smart_contract(self, code):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        smart_contract = SmartContract(code)
        new_block = Block(index, previous_hash, timestamp, code, "")
        validator = self.pos.select_validator()
        print(f"Smart Contract Block #{index} created by {validator} using PoS")
        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)
        block_hash = new_block.hash
        self.smart_contracts[block_hash] = smart_contract
        return block_hash

    def execute_smart_contract_from_block(self, block_hash, **kwargs):
        if block_hash in self.smart_contracts:
            smart_contract = self.smart_contracts[block_hash]
            smart_contract.execute(self, **kwargs)
        else:
            print("Smart contract not found in blockchain")

    def mint_nft(self, owner, metadata):
        tokenId = self.nft_contract.mint(owner, metadata)
        data = json.dumps({
            "type": "mint",
            "owner": owner,
            "tokenId": tokenId,
            "metadata": metadata
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_nft(self, from_, to, tokenId):
        self.nft_contract.transfer(from_, to, tokenId)
        data = json.dumps({
            "type": "transfer",
            "from": from_,
            "to": to,
            "tokenId": tokenId
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def mint_fungible_token(self, to, amount):
        self.fungible_token_contract.mint(to, amount)
        data = json.dumps({
            "type": "mint_fungible",
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_fungible_token(self, from_, to, amount):
        self.fungible_token_contract.transfer(from_, to, amount)
        data = json.dumps({
            "type": "transfer_fungible",
            "from": from_,
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

# Demo Proof of Stake

In [54]:
# Contoh Penggunaan
pos = ProofOfStake()
my_blockchain = MyBlockchain(pos)

In [55]:
# Add stakeholders using the ProofOfStake object
pos.add_stakeholder('Alice', 10)
pos.add_stakeholder('Bob', 20)
pos.add_stakeholder('Charlie', 30)

In [56]:
# Menambahkan blok biasa
my_blockchain.add_block('Block 1 Data')
my_blockchain.add_block('Block 2 Data')

Block #17 created by Charlie using PoS
Block #18 created by Bob using PoS


In [57]:
# Menambahkan smart contract
smart_contract_code = """
def contract_function(storage):
    storage['counter'] = storage.get('counter', 0) + 1
    print(f"Counter value: {storage['counter']}")
contract_function(storage)
"""

In [58]:
block_hash = my_blockchain.store_smart_contract(smart_contract_code)
print(f"Smart contract stored in block with hash: {block_hash}")

Smart Contract Block #19 created by Charlie using PoS
Smart contract stored in block with hash: 8169f22486e81087472f5ea110a8e1c06308dde96b754b09a1a74e9dd9ba2572


In [59]:
# Menjalankan smart contract dari blockchain
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 1


In [60]:
# Menjalankan kembali smart contract untuk melihat perubahan state
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 2


In [61]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage):\n    st

In [62]:
# Membuat NFT baru
owner_address = "0xfeedfacefeedfacefeedfacefeedfacefeedface"
asset_uri = "https://example2.co.id"
mint_block_hash = my_blockchain.mint_nft(owner_address, asset_uri)
print(f"NFT minted in block with hash: {mint_block_hash}")

Block #20 created by Charlie using PoS
NFT minted in block with hash: 6dd57ebc234812e569a6eb2afaa776f21953c804aebe370cb396a8f46ab65d0c


In [63]:
# Transfer NFT
new_owner_address = "0xbadc0ffeebadc0ffeebadc0ffeebadc0ffeebadc"
token_id = hashlib.sha256(asset_uri.encode()).hexdigest()
transfer_block_hash = my_blockchain.transfer_nft(owner_address, new_owner_address, token_id)
print(f"NFT transferred in block with hash: {transfer_block_hash}")

Block #21 created by Bob using PoS
NFT transferred in block with hash: 9d71658f0eb1bf793fc2f185eabfd6d4da5819abd51eace69a581433ff80a3fd


In [64]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage):\n    st

In [65]:
# Demonstrasi Token Fungible (ERC20)
my_blockchain.mint_fungible_token("0xabc", 1000)
my_blockchain.mint_fungible_token("0xdef", 500)

Block #22 created by Charlie using PoS
Block #23 created by Charlie using PoS


'e1a314c9c9b429d23c3ef71d44a29c4061dc5371347db76e88b5f17457fe1a1d'

In [66]:
print("Balance of 0xabc:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc: 1000
Balance of 0xdef: 500


In [67]:
my_blockchain.transfer_fungible_token("0xabc", "0xdef", 200)

Block #24 created by Charlie using PoS


'e17da777ececc0464bf8bd3704d56022b310c84d098fae62d35c66d90b5d0576'

In [68]:
print("Balance of 0xabc after transfer:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef after transfer:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc after transfer: 800
Balance of 0xdef after transfer: 700


In [69]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage):\n    st

# Demo Proof of Burn

In [70]:
pob = ProofOfBurn()

In [71]:
pob.burn_coins('Alice', 10)
pob.burn_coins('Bob', 20)
pob.burn_coins('Charlie', 30)

In [72]:
blockchain_pob = Blockchain(pob)
blockchain_pob.add_block('Block 1 Data')
blockchain_pob.add_block('Block 2 Data')

Block #25 created by Charlie using PoB
Block #26 created by Alice using PoB


In [73]:
print("Blockchain content with PoB in JSON:")
print(blockchain_pob.to_json())

Blockchain content with PoB in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage)

# Demo Proof of Elapsed Time

In [74]:
poet = ProofOfElapsedTime()
poet.wait_times = {'Alice': 0, 'Bob': 0, 'Charlie': 0}

In [75]:
blockchain_poet = Blockchain(poet)
blockchain_poet.add_block('Block 1 Data')
blockchain_poet.add_block('Block 2 Data')

Block #27 created by Alice using PoET
Block #28 created by Charlie using PoET


In [76]:
print("Blockchain content with PoET in JSON:")
print(blockchain_poet.to_json())

Blockchain content with PoET in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage

# Demo Proof of Byzantine Fault Tolerance

In [77]:
validators = ["Alice", "Bob", "Charlie", "David"]
faulty_validators = ["Charlie"]

bft = ByzantineFaultTolerance(validators, faulty_validators)

In [78]:
blockchain_bft = Blockchain(bft)
blockchain_bft.add_block('Block 1 Data')
blockchain_bft.add_block('Block 2 Data')

Block #29 created by Alice using BFT
Block #30 created by David using BFT


In [79]:
print("Blockchain content with BFT in JSON:")
print(blockchain_bft.to_json())

Blockchain content with BFT in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage)

# Demo DAO untuk Grant DAOs

In [80]:
class GrantDAOs(MyBlockchain):
    def __init__(self,pos):
        super().__init__(pos)
        self.grants = {}

    def create_grant(self, grant_id, proposer, amount, description):
        self.grants[grant_id] = {
            'proposer': proposer,
            'amount': amount,
            'description': description,
            'votes': {'yes': 0, 'no': 0},
            'executed': False
        }
        data = json.dumps({
            "type": "create_grant",
            "grant_id": grant_id,
            "proposer": proposer,
            "amount": amount,
            "description": description
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def vote_grant(self, grant_id, stakeholder, vote):
        if grant_id in self.grants:
            if vote not in ['yes', 'no']:
                raise ValueError("Vote must be 'yes' atau 'no'")
            self.grants[grant_id]['votes'][vote] += 1
            data = json.dumps({
                "type": "vote_grant",
                "grant_id": grant_id,
                "stakeholder": stakeholder,
                "vote": vote
            })
            self.add_block(data)
            return self.get_latest_block().hash
        else:
            raise ValueError("Grant ID tidak ditemukan")

    def execute_grant(self, grant_id):
        if grant_id in self.grants:
            grant = self.grants[grant_id]
            if grant['votes']['yes'] > grant['votes']['no'] and not grant['executed']:
                self.reward_player(grant['proposer'], grant['amount'])
                grant['executed'] = True
                data = json.dumps({
                    "type": "execute_grant",
                    "grant_id": grant_id
                })
                self.add_block(data)
                return self.get_latest_block().hash
            else:
                raise ValueError("Grant belum disetujui atau sudah dieksekusi")
        else:
            raise ValueError("Grant ID tidak ditemukan")

    def reward_player(self, player, amount):
        self.fungible_token_contract.mint(player, amount)
        data = json.dumps({
            "type": "reward_player",
            "player": player,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

In [81]:
# Demonstrasi Penggunaan
pos = ProofOfStake()
grant_daos = GrantDAOs(pos)

pos.add_stakeholder('Alice', 10)
pos.add_stakeholder('Bob', 20)
pos.add_stakeholder('Charlie', 30)
pos.add_stakeholder('Herman', 15)
pos.add_stakeholder('Hani', 7)
pos.add_stakeholder('Dolbi', 50)

# Menambahkan blok biasa
grant_daos.add_block('Block 1 Data')
grant_daos.add_block('Block 2 Data')

Block #31 created by Alice using PoS
Block #32 created by Herman using PoS


In [82]:
# Membuat proposal grant
grant_id = "grant1"
proposer = "Alice"
amount = 100
description = "Research on sustainable blockchain solutions"
create_grant_hash = grant_daos.create_grant(grant_id, proposer, amount, description)
print(f"Grant proposal created in block with hash: {create_grant_hash}")

Block #33 created by Dolbi using PoS
Grant proposal created in block with hash: 0378b9a984169b3bf821b3f35a37774717879189c094ac93e609d9f69dba45ec


In [83]:
# Stakeholder voting
vote_hash_1 = grant_daos.vote_grant(grant_id, 'Bob', 'yes')
print(f"Grant vote by Bob in block with hash: {vote_hash_1}")

vote_hash_2 = grant_daos.vote_grant(grant_id, 'Charlie', 'no')
print(f"Grant vote by Charlie in block with hash: {vote_hash_2}")

vote_hash_3 = grant_daos.vote_grant(grant_id, 'Hani', 'no')
print(f"Grant vote by Hani in block with hash: {vote_hash_3}")

Block #34 created by Dolbi using PoS
Grant vote by Bob in block with hash: 1e86be4d5f24e836004554851d3cac4a63a1b36aa0f8647095c7ebfdf15a0cc6
Block #35 created by Dolbi using PoS
Grant vote by Charlie in block with hash: 62df72959fb4d06956b7119febb6842d834e31bfb0e4ff76460707ce0923654c
Block #36 created by Herman using PoS
Grant vote by Hani in block with hash: f2995e7fab4efd32e0c33e1f95891e2e8fbfb7be567255d24e7cef22f1969126


In [84]:
# Eksekusi grant setelah voting
try:
    execute_grant_hash = grant_daos.execute_grant(grant_id)
    print(f"Grant executed in block with hash: {execute_grant_hash}")
except ValueError as e:
    print(f"Error executing grant: {e}")
    # Check the votes to see if the grant was approved
    grant = grant_daos.grants.get(grant_id)
    if grant:
        print(f"Grant votes: {grant['votes']}")

Error executing grant: Grant belum disetujui atau sudah dieksekusi
Grant votes: {'yes': 1, 'no': 2}


In [85]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(grant_daos.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage):\n    st

# Demo DAO untuk MediaDAOs

In [86]:
class MediaDAOs(MyBlockchain):
    def __init__(self,pos):
        super().__init__(pos)
        self.proposals = {}

    def create_proposal(self, proposal_id, proposer, description):
        self.proposals[proposal_id] = {
            'proposer': proposer,
            'description': description,
            'votes': {'yes': 0, 'no': 0},
            'executed': False
        }
        data = json.dumps({
            "type": "create_proposal",
            "proposal_id": proposal_id,
            "proposer": proposer,
            "description": description
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def vote_proposal(self, proposal_id, stakeholder, vote):
        if proposal_id in self.proposals:
            if vote not in ['yes', 'no']:
                raise ValueError("Vote must be 'yes' or 'no'")
            self.proposals[proposal_id]['votes'][vote] += 1
            data = json.dumps({
                "type": "vote_proposal",
                "proposal_id": proposal_id,
                "stakeholder": stakeholder,
                "vote": vote
            })
            self.add_block(data)
            return self.get_latest_block().hash
        else:
            raise ValueError("Proposal ID not found")

    def execute_proposal(self, proposal_id):
        if proposal_id in self.proposals:
            proposal = self.proposals[proposal_id]
            if proposal['votes']['yes'] > proposal['votes']['no'] and not proposal['executed']:
                # Logic to reward or execute the proposal
                proposal['executed'] = True
                data = json.dumps({
                    "type": "execute_proposal",
                    "proposal_id": proposal_id
                })
                self.add_block(data)
                return self.get_latest_block().hash
            else:
                raise ValueError("Proposal not approved or already executed")
        else:
            raise ValueError("Proposal ID not found")

In [87]:
# Demonstrasi Penggunaan
pos = ProofOfStake()
media_daos = MediaDAOs(pos)

pos.add_stakeholder('Alice', 10)
pos.add_stakeholder('Bob', 20)
pos.add_stakeholder('Charlie', 30)
pos.add_stakeholder('Hani', 25)

In [88]:
# Menambahkan blok biasa
media_daos.add_block('Block 1 Data')
media_daos.add_block('Block 2 Data')

Block #37 created by Charlie using PoS
Block #38 created by Alice using PoS


In [89]:
# Membuat proposal media
proposal_id = "proposal1"
proposer = "Alice"
description = "Create a documentary on blockchain technology"
create_proposal_hash = media_daos.create_proposal(proposal_id, proposer, description)
print(f"Media proposal created in block with hash: {create_proposal_hash}")

Block #39 created by Charlie using PoS
Media proposal created in block with hash: f40ff3a556dd656b8167daeb93201c55061336151d892a1a426a445189676a90


In [90]:
# Stakeholder voting
vote_hash_1 = media_daos.vote_proposal(proposal_id, 'Bob', 'yes')
print(f"Proposal vote by Bob in block with hash: {vote_hash_1}")

vote_hash_2 = media_daos.vote_proposal(proposal_id, 'Charlie', 'yes')
print(f"Proposal vote by Charlie in block with hash: {vote_hash_2}")

vote_hash_3 = media_daos.vote_proposal(proposal_id, 'Hani', 'yes')
print(f"Proposal vote by Hani in block with hash: {vote_hash_3}")

Block #40 created by Bob using PoS
Proposal vote by Bob in block with hash: 37d959954732ba12d3dde4395ebfbb8f0a92158fed9354070e8988add9a0b42d
Block #41 created by Hani using PoS
Proposal vote by Charlie in block with hash: f1d88269c7aa656e0a2d9d66b6a3fd8147ff620d90108ec4191bc29ffbf08411
Block #42 created by Bob using PoS
Proposal vote by Hani in block with hash: 742b7a853d3ee75ab8fe8bb236b0f82700a67efbe29662eb1ebf80e30f98abf6


In [91]:
# Eksekusi proposal setelah voting
try:
    execute_proposal_hash = media_daos.execute_proposal(proposal_id)
    print(f"Proposal executed in block with hash: {execute_proposal_hash}")
except ValueError as e:
    print(f"Error executing proposal: {e}")
    # Check the votes to see if the proposal was approved
    proposal = media_daos.proposals.get(proposal_id)
    if proposal:
        print(f"Grant votes: {proposal['votes']}")

Block #43 created by Charlie using PoS
Proposal executed in block with hash: ef5843eb83a5cc47c998f34686509896c45ac8f22caf36d6c452917e64335478


In [92]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(media_daos.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719939711,
        "data": "Genesis Block",
        "hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e"
    },
    {
        "index": 1,
        "previous_hash": "beb899225810bb5ca71c48a68ad5c49ad7f0ee47bb1d5643a2f2421e06b0927e",
        "timestamp": 1719939743,
        "data": "Block 1 Data",
        "hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a"
    },
    {
        "index": 2,
        "previous_hash": "ac5648b81b103ea49088d0125b8a4f0659c530215e9933a36a747e818f07216a",
        "timestamp": 1719939743,
        "data": "Block 2 Data",
        "hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551"
    },
    {
        "index": 3,
        "previous_hash": "3fff92759bf115540a8ad92374e18d433da01eef2e262d3c579e784ef5d98551",
        "timestamp": 1719939754,
        "data": "\ndef contract_function(storage):\n    st